In [1]:
import os

import numpy as np
import pandas as pd
import finpy_tse as tse
import time
from persiantools.jdatetime import JalaliDate
from tqdm import tqdm
import requests

In [2]:
def get_path():
    try:
        path = input("Enter your Destination file location :")
        os.listdir(path)
    except Exception as e:
        with open('errors.txt', 'a') as f:
            f.write(f'{e.__cause__}\n')
    return path

In [3]:
def internet_connection():
    try:
        response = requests.get("https://www.google.com", timeout=5)
        return True
    except requests.ConnectionError:
        return False

In [8]:
def get_method():
    inp = input("Enter your get method (all=1,select=2,enter=3) :")
    if inp in ['1', '2', '3']:
        return inp
    else:
        get_method()

In [9]:
def create_log():
    if 'log.txt' in os.listdir():
        with open('log.txt', 'r') as f:
            if f.read() == '':
                f.close()
                with open('log.txt', 'w+') as f:
                    f.write(f'{0},{time.localtime(time.time())[:3]},{JalaliDate.today()}\n')
    else:
        with open('log.txt', 'w') as f:
            f.write(f'{0},{time.localtime(time.time())[:3]},{JalaliDate.today()}\n')

In [10]:
def get_data():
    create_log()
    with open('log.txt', 'r') as f:
        msg = f.readlines()[-1].split(',')

    def first_export_data(inp_symbols):
        for i in tqdm(inp_symbols.index):
            get_df = tse.get_price_history(stock=i, ignore_date=True, double_date=True)
            get_df = get_df.loc[:, ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
            get_df.Date = np.array([str(i)[:10].replace('-', '') for i in get_df.Date])
            get_df.reset_index(inplace=True)
            get_df.drop('J-Date', axis=1, inplace=True)
            get_df.rename(
                columns={'Date': '<DTYYYYMMDD>', 'Open': '<Open>', 'High': '<HIGH>', 'Low': '<LOW>', 'Close': '<CLOSE>',
                         'Volume': '<VOL>'}, inplace=True)
            get_df.to_csv(get_path() + '\\' + f"{inp_symbols['Name(EN)'][i]}".replace(' ', '_') + ".prn")

    def second_export_data(inp_symbols):
        for i in tqdm(inp_symbols.index):
            get_df = tse.get_price_history(stock=i, start_date=msg[-1].replace('\n', ''), double_date=True)
            get_df = get_df.loc[:, ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
            get_df.Date = np.array([str(i)[:10].replace('-', '') for i in get_df.Date])
            get_df.reset_index(inplace=True)
            get_df.drop('J-Date', axis=1, inplace=True)
            get_df.rename(
                columns={'Date': '<DTYYYYMMDD>', 'Open': '<Open>', 'High': '<HIGH>', 'Low': '<LOW>',
                         'Close': '<CLOSE>',
                         'Volume': '<VOL>'}, inplace=True)
            df = pd.read_csv(get_path() + '\\' + f"{inp_symbols['Name(EN)'][i]}".replace(' ', '_') + ".prn")
            get_df = pd.concat([df, get_df], axis=0)
            get_df.to_csv(get_path() + '\\' + f"{inp_symbols['Name(EN)'][i]}".replace(' ', '_') + ".prn")

    def select_symbols(inp_symbols):
        select_symbol = []
        for i in symbols.index:
            select = input(f'if you want this ticker(symbol) -> {i} <- enter 1 else enetr 0:')
            if select == '1':
                select_symbol.append(i)
            elif select == 'end':
                break
        return inp_symbols.loc[select_symbol, :]

    def enter_symbols(inp_symbols):
        enter_symbol = []
        while True:
            inp = input("Enter your Ticker(symbol) :")
            if inp in symbols.index:
                enter_symbol.append(inp)
            else:
                break
        return inp_symbols.loc[enter_symbol, :]

    if msg[0] == '0':
        symbols = tse.Build_Market_StockList(
            bourse=True,
            farabourse=True,
            payeh=True,
            detailed_list=True,
            show_progress=True, save_path='symbols.xlsx')
        method = get_method()
        if method == '1':
            first_export_data(symbols)

        elif method == '2':
            selected = select_symbols(symbols)
            selected.to_excel('select_symbols.xlsx')
            selected = selected['Name(EN)']
            first_export_data(selected)

        elif method == '3':
            entered = enter_symbols(symbols)
            entered.to_excel("enter_symbols.xlsx")
            entered = entered["Name(EN)"]
            first_export_data(entered)
        with open('log.txt', 'a') as f:
            f.write(f'{1},{time.localtime(time.time())[:3]},{JalaliDate.today()}\n')
    else:
        symbols = pd.read_excel('symbols.xlsx')
        inp = input('if you want continue same as past enter 1 else 0 :')
        if inp == '1':
            if 'select_symbols.xlsx' or 'enter_symbols.xlsx' not in os.listdir():
                uti_symbols = symbols["Name(EN)"]
                second_export_data(uti_symbols)
            elif 'select_symbols.xlsx' in os.listdir():
                uti_symbols = pd.read_excel('select_symbols.xlsx')["Name(EN)"]
                second_export_data(uti_symbols)
            elif 'enter_symbols.xlsx' in os.listdir():
                uti_symbols = pd.read_excel('enter_symbols.xlsx')["Name(EN)"]
                second_export_data(uti_symbols)
            with open('log.txt', 'a') as f:
                f.write(f'{int(msg[0]) + 1},{time.localtime(time.time())[:3]},{JalaliDate.today()}\n')
        elif inp == '0':
            try:
                os.rmdir('select_symbols.xlsx')
                os.rmdir('enter_symbols.xlsx')
            except:
                pass
            method = get_method()
            if method == '1':
                second_export_data(symbols)
            elif method == '2':
                selected = select_symbols(symbols)
                selected.to_excel('select_symbols.xlsx')
                selected = selected['Name(EN)']
                second_export_data(selected)
            elif method == '3':
                entered = enter_symbols(symbols)
                entered.to_excel("enter_symbols.xlsx")
                entered = entered["Name(EN)"]
                second_export_data(entered)
            with open('log.txt', 'a') as f:
                f.write(f'{int(msg[0]) + 1},{time.localtime(time.time())[:3]},{JalaliDate.today()}\n')

In [11]:
if internet_connection():

    get_data()

Progress : 100 % , Done in 31 seconds!
Save path does not exist, you can handle saving this data by returned dataframe as Excel using ".to_excel()", if you will!
Save path does not exist, you can handle saving this data by returned dataframe as CSV using ".to_csv()", if you will!


  0%|          | 0/5 [00:28<?, ?it/s]


KeyError: 'Name(EN)'